In [1]:
import requests

In [2]:
standings_url = 'https://fbref.com/en/comps/12/Premier-League-Stats'

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [6]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [7]:
data = requests.get(team_urls[0])

In [8]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [9]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [10]:
data = requests.get(f"https://fbref.com{links[0]}")

In [11]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [12]:
shooting

For Barcelona                                                             \
            Date   Time          Comp        Round  Day Venue Result  GF GA   
0     2022-08-13  21:00       La Liga  Matchweek 1  Sat  Home      D   0  0   
1     2022-08-21  22:00       La Liga  Matchweek 2  Sun  Away      W   4  1   
2     2022-08-28  19:30       La Liga  Matchweek 3  Sun  Home      W   4  0   
3     2022-09-03  21:00       La Liga  Matchweek 4  Sat  Away      W   3  0   
4     2022-09-07  21:00  Champions Lg  Group stage  Wed  Home      W   5  1   
5     2022-09-10  18:30       La Liga  Matchweek 5  Sat  Away      W   4  0   
6     2022-09-13  21:00  Champions Lg  Group stage  Tue  Away      L   0  2   
7     2022-09-17  16:15       La Liga  Matchweek 6  Sat  Home      W   3  0   
8     2022-10-01  21:00       La Liga  Matchweek 7  Sat  Away      W   1  0   
9     2022-10-04  21:00  Champions Lg  Group stage  Tue  Away      L   0  1   
10           NaN    NaN           NaN          NaN  NaN   NaN  7-1-2  24  5   

                       ... Standard             Expected                     \
             Opponent  ...     Dist FK PK PKatt       xG  npxG npxG/Sh G-xG   
0      Rayo Vallecano  ...     16.6  1  0     0      2.1   2.1    0.10 -2.1   
1       Real Sociedad  ...     13.8  0  0     0      2.2   2.2    0.15  1.8   
2          Valladolid  ...     13.9  1  0     0      2.9   2.9    0.12  1.1   
3             Sevilla  ...     16.0  2  0     0      3.2   3.2    0.18 -0.2   
4   cz Viktoria Plzeň  ...     15.5  0  0     0      2.0   2.0    0.10  3.0   
5               Cádiz  ...     15.0  0  0     0      3.0   3.0    0.19  1.0   
6    de Bayern Munich  ...     15.4  1  0     0      2.0   2.0    0.11 -2.0   
7               Elche  ...     14.8  0  0     0      4.1   4.1    0.17 -1.1   
8            Mallorca  ...     19.2  0  0     0      0.5   0.5    0.05  0.5   
9            it Inter  ...     12.9  0  0     0      0.6   0.6    0.08 -0.6   
10                NaN  ...     15.3  5  0     0     22.7  22.7    0.13  1.3   

           Unnamed: 25_level_0  
   np:G-xG        Match Report  
0     -2.1        Match Report  
1      1.8        Match Report  
2      1.1        Match Report  
3     -0.2        Match Report  
4      3.0        Match Report  
5      1.0        Match Report  
6     -2.0        Match Report  
7     -1.1        Match Report  
8      0.5        Match Report  
9     -0.6        Match Report  
10     1.3                 NaN  

[11 rows x 26 columns]

In [13]:
shooting.columns = shooting.columns.droplevel()

In [14]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [15]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-08-13,21:00,La Liga,Matchweek 1,Sat,Home,D,0.0,0.0,Rayo Vallecano,...,4-3-3,Alejandro Hernández,Match Report,NaN,21,5,16.6,1,0,0
1,2022-08-21,22:00,La Liga,Matchweek 2,Sun,Away,W,4.0,1.0,Real Sociedad,...,3-4-3,José Luis Munuera,Match Report,NaN,15,7,13.8,0,0,0
2,2022-08-28,19:30,La Liga,Matchweek 3,Sun,Home,W,4.0,0.0,Valladolid,...,4-3-3,Ricardo de Burgos,Match Report,NaN,25,9,13.9,1,0,0
3,2022-09-03,21:00,La Liga,Matchweek 4,Sat,Away,W,3.0,0.0,Sevilla,...,4-3-3,Antonio Matéu Lahoz,Match Report,NaN,19,6,16.0,2,0,0
4,2022-09-07,21:00,Champions Lg,Group stage,Wed,Home,W,5.0,1.0,cz Viktoria Plzeň,...,4-3-3,Lawrence Visser,Match Report,NaN,20,10,15.5,0,0,0


In [16]:
team_data.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain', 'Formation',
       'Referee', 'Match Report', 'Notes', 'Sh', 'SoT', 'Dist', 'FK', 'PK',
       'PKatt'],
      dtype='object')

In [17]:
years = list(range(2022, 2019, -1))
all_matches = []

In [18]:
standings_url = 'https://fbref.com/en/matches/'

In [19]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

KeyError: "['FK'] not in index"

In [ ]:
len(all_matches)

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

In [ ]:
# match_df.to_csv("matches.csv")